In [1]:
import h5py
import rasterio
from rasterio.mask import mask
from rasterio.transform import from_origin
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd 
import pandas as pd
from dnb_annual import *
from variables import years, composites, region_map, region_names

c:\Users\jakub\anaconda\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\jakub\anaconda\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [3]:
# this script is used only once to generate the regional images for each year
# country_polygons = gpd.read_file("geoBoundaries-UKR-ADM1.geojson")

# for year in years:
#     dnb = dnb_annual(year, composites, country_polygons)
#     dnb.load_all_data()
#     dnb.save_rasters()
#     dnb.load_rasters()
#     dnb.build_regional_images()
#     dnb.add_padding()
#     dnb.save_regional_images()

In [2]:
# this script is used to clean gdp data

# Inflation data
# inflation = pd.read_excel("data/isc_reg.xls", skiprows=2, header=1)
# inflation = inflation.drop(columns=inflation.columns[0])
# inflation = inflation.rename(columns={inflation.columns[-1]: "region"})
# inflation = inflation[~inflation["region"].isin(["Ukraine", "oblasts"])]
# inflation = inflation.dropna()
# inflation["region"] = inflation["region"].map(region_map)
# inflation.columns = inflation.columns.astype(str)
# inflation = inflation.melt(id_vars="region", var_name="year", value_name="inflation")
# inflation.to_csv("data/inflation.csv", index=False)

# GDP data
gdp = pd.read_excel("data/ukr_reg_gdp.xls", skiprows=3, header=1)
gdp = gdp.drop(columns=gdp.columns[0])
gdp = gdp.iloc[:, np.r_[18:36, -1]]
gdp = gdp.rename(columns={gdp.columns[-1]: "region"})
gdp = gdp[~gdp["region"].isin(["Ukrane", "oblasts"])]
gdp = gdp.dropna()
gdp["region"] = gdp["region"].map(region_map)
gdp["region"] = gdp["region"].fillna("Sevastopol")
gdp.columns = gdp.columns.astype(str)
gdp = gdp.rename(columns={gdp.columns[i]: gdp.columns[i][:4] for i in range(18)})
gdp = gdp.melt(id_vars="region", var_name="year", value_name="real_gdp_change")

# include only years from 2012 inclusive, exclude Sevastopol and the Autonomous Republic of Crimea
gdp = gdp[gdp["year"].astype(int) >= 2012]
gdp = gdp[~gdp["region"].isin(["Sevastopol", "Autonomous Republic of Crimea"])]

# set the value for the starting year to 100 (2012), NaN for the rest
gdp.loc[gdp["year"] == "2012", "real_gdp"] = 100
gdp = gdp.sort_values(by=["region", "year"])
gdp["real_gdp_change"] = gdp["real_gdp_change"] / 100

# reste the index
gdp = gdp.reset_index(drop=True)

# # calculate the real gdp
for i in range(1, gdp.shape[0]):

    # skip if the year is 2012
    if gdp.loc[gdp.index[i], "year"] == "2012":
        continue
    else:
        gdp.loc[gdp.index[i], "real_gdp"] = gdp.loc[gdp.index[i-1], "real_gdp"] * (gdp.loc[gdp.index[i], "real_gdp_change"])

# delete the real_gdp_change column
gdp = gdp.drop(columns="real_gdp_change")

# get the nominal gdp
gdp_nominal = pd.read_excel("data/ukr_reg_gdp.xls", skiprows=3, header=1)
gdp_nominal = gdp_nominal.iloc[:, np.r_[9, -1]]
gdp_nominal.columns = ["gdp_nominal", "region"]
gdp_nominal = gdp_nominal[~gdp_nominal["region"].isin(["Ukrane", "oblasts"])]
gdp_nominal = gdp_nominal.dropna()
gdp_nominal["region"] = gdp_nominal["region"].map(region_map)
gdp_nominal["region"] = gdp_nominal["region"].fillna("Sevastopol")

# merge nominal gdp to real gdp by region
gdp = gdp.merge(gdp_nominal, on="region")

# multiple the real gdp by the nominal gdp
gdp["real_gdp"] = gdp["real_gdp"] * gdp["gdp_nominal"]

# drop the nominal gdp column
gdp = gdp.drop(columns="gdp_nominal")

# for the region column, change all spaces to _
gdp["region"] = gdp["region"].str.replace(" ", "_")

# save the data
gdp.to_csv("data/clean_gdp.csv", index=False)


### Neural Network

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Resizing, Dropout, BatchNormalization
from keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler


# # Loading the MNIST dataset
# from keras.datasets import mnist
# (train_images, train_labels), (test_images, test_labels) = mnist.load_data()

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

SystemError: initialization of _pywrap_checkpoint_reader raised unreported exception

In [70]:
# load clean gdp data
gdp = pd.read_csv("data/clean_gdp.csv")

# Initialise a three dimensional array to store the images with the shape (number of images, height, width, channels)
X = np.zeros((len(gdp), 765, 1076, 4))
y = np.zeros(len(gdp))

# load the snow covered and snow free images, add them together and append to the list
for i in range(len(gdp)):

    # get year, region, and gdp
    year = gdp["year"][i]
    region = gdp["region"][i]
    gdp_value = gdp["real_gdp"][i]

    # get the file name
    file_name = f"{year}_{region}.h5"

    # load the image
    file_path = f"data/annual_region_images/{file_name}"
    
    with h5py.File(file_path, 'r') as annual_region:
        nearnad_snow_cov = annual_region["NearNadir_Composite_Snow_Covered"][:]
        nearnad_snow_free = annual_region["NearNadir_Composite_Snow_Free"][:]
        offnad_snow_cov = annual_region["OffNadir_Composite_Snow_Covered"][:]
        offnad_snow_free = annual_region["OffNadir_Composite_Snow_Free"][:]
        allangle_snow_cov = annual_region["AllAngle_Composite_Snow_Covered"][:]
        allangle_snow_free = annual_region["AllAngle_Composite_Snow_Free"][:]

        # add the two images together
        # combined = snow_covered + snow_free

    # add the gdp value to y
    y[i] = gdp_value

    # append both images as two channels to to X
    X[i, :, :, 0] = allangle_snow_cov
    X[i, :, :, 1] = allangle_snow_free
    # X[i, :, :, 2] = offnad_snow_cov
    # X[i, :, :, 3] = offnad_snow_free
    # X[i, :, :, 4] = nearnad_snow_cov
    # X[i, :, :, 5] = nearnad_snow_free
# print(X.shape)
# print(y.shape)

# Normalise the images
maximum = X.max()
X = X / maximum

# standardise gdp values
y_mean = y.mean()
y_std = y.std()
y = (y - y_mean) / y_std

# print(y.mean())
# print(maximum)

# change the format to a float16
X = X.astype(np.float16)
y = y.astype(np.float16)

MemoryError: Unable to allocate 6.13 GiB for an array with shape (250, 765, 1076, 4) and data type float64

In [49]:
# select 80% of the data for training, choose randomly
# X is the images, y is the gdp
train_size = int(0.8 * len(gdp))
test_size = len(gdp) - train_size

# select randomly train_size numbers from 0 to len(gdp)
train_indices = np.random.choice(len(gdp), train_size, replace=False)
test_indices = np.setdiff1d(np.arange(len(gdp)), train_indices)

# get the train data
X_train = X[train_indices]
y_train = y[train_indices]

# get the test data
X_test = X[test_indices]
y_test = y[test_indices]


In [50]:
model = Sequential()
# Resizing the images
model.add(Resizing(300, 440, input_shape=(765, 1076, 2)))
# Start with Convolutional layers
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))  # Additional Conv layer
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))  # Additional Conv layer
model.add(MaxPooling2D((2, 2)))
# Flatten the results to feed into a dense layer
model.add(Flatten())
# Add dense layers (hidden layers)
model.add(Dense(128, activation='relu'))  # Upscaled dense layer
model.add(Dense(64, activation='relu'))   # Additional dense layer
# Output layer
model.add(Dense(1))
# Compile the model
model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mae'])

In [12]:
model = Sequential()
# Resizing the images
model.add(Resizing(300, 440, input_shape=(765, 1076, 4)))

# Start with Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu'))  # Increased number of filters
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))  # Increased number of filters
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))  # Increased number of filters
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))  # Increased number of filters
model.add(MaxPooling2D((2, 2)))

# Flatten the results to feed into a dense layer
model.add(Flatten())

# Add dense layers (hidden layers)
model.add(Dense(256, activation='relu'))  # Increased the number of neurons
model.add(Dense(128, activation='relu'))  # Increased the number of neurons
model.add(Dense(64, activation='relu'))   # Kept as is for detailed feature extraction

# Output layer
model.add(Dense(1))

# Compile the model
model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mae'])

In [51]:
# check the size of X_train and y_train
print(X_train.shape)
print(X_train[1, :, :, 0].max())

(200, 765, 1076, 4)
0.0731


In [52]:
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)  # Assuming you have a validation split of 20%


Epoch 1/20
5/5 [==============================] - 19s 2s/step - loss: 0.9402 - mae: 0.6076 - val_loss: 1.2130 - val_mae: 0.6471
Epoch 2/20
5/5 [==============================] - 13s 3s/step - loss: 0.9138 - mae: 0.5837 - val_loss: 1.1034 - val_mae: 0.6271
Epoch 3/20
5/5 [==============================] - 11s 2s/step - loss: 0.7788 - mae: 0.5462 - val_loss: 1.0158 - val_mae: 0.5973
Epoch 4/20
5/5 [==============================] - 13s 3s/step - loss: 0.6202 - mae: 0.5160 - val_loss: 0.7500 - val_mae: 0.5055
Epoch 5/20
5/5 [==============================] - 12s 2s/step - loss: 0.3906 - mae: 0.3476 - val_loss: 0.3057 - val_mae: 0.3249
Epoch 6/20
5/5 [==============================] - 11s 2s/step - loss: 0.1644 - mae: 0.2705 - val_loss: 0.1656 - val_mae: 0.2421
Epoch 7/20
5/5 [==============================] - 11s 2s/step - loss: 0.1024 - mae: 0.2111 - val_loss: 0.0568 - val_mae: 0.1605
Epoch 8/20
5/5 [==============================] - 10s 2s/step - loss: 0.0882 - mae: 0.1786 - val_loss: 0

In [53]:
# Step 6: Evaluate your model on the testing data
test_loss, test_mae = model.evaluate(X_test, y_test)
print('Test MAE:', test_mae) # mean absolute error
print('Test Loss:', test_loss)

2/2 [==============================] - 1s 202ms/step - loss: 0.1087 - mae: 0.1321
Test MAE: 0.1321280300617218
Test Loss: 0.10873400419950485


In [69]:
y_hat = model.predict(X_test).flatten()

# print the mean absolute percentage error
print(np.mean(100*np.abs((y_test - y_hat) / y_test)))

2/2 [==============================] - 3s 231ms/step
109.363976


In [9]:
# get the predictions from X_test
y_hat = model.predict(X_test).flatten()

# convert the predictions back to the original scale
# y_hat = y_hat * y_std + y_mean
# y_test = y_test * y_std + y_mean

# compute the mae
mae = np.mean(np.abs(y_test - y_hat))
print(mae)

# compute the mean percentage error
percentage_error = np.mean(100*np.abs((y_test - y_hat) / y_test))
print(percentage_error)


2/2 [==============================] - 0s 56ms/step
0.10076753842329474
36.946041487261475


## Recurrent NN

In [ ]:
# Preprocess the data

# load clean gdp data
gdp = pd.read_csv("data/clean_gdp.csv")

# Initialise a three dimensional array to store the images with the shape (number of images, height, width, channels)
X = np.zeros((len(gdp), 765, 1076, 4))
y = np.zeros(len(gdp))

## Random Forest and other

In [30]:
# load the tabular data
data = pd.read_csv("data/tabular_data_ukraine.csv")

# turn the region column into a categorical variable using one hot encoding
data = pd.get_dummies(data, columns=["region"])


In [31]:
data.columns

Index(['year', 'gdp', 'nearnad_snow_cov_1', 'nearnad_snow_cov_2',
       'nearnad_snow_cov_3', 'nearnad_snow_cov_4', 'nearnad_snow_cov_5',
       'nearnad_snow_cov_6', 'nearnad_snow_cov_7', 'nearnad_snow_cov_8',
       'nearnad_snow_cov_9', 'nearnad_snow_cov_10', 'nearnad_snow_free_1',
       'nearnad_snow_free_2', 'nearnad_snow_free_3', 'nearnad_snow_free_4',
       'nearnad_snow_free_5', 'nearnad_snow_free_6', 'nearnad_snow_free_7',
       'nearnad_snow_free_8', 'nearnad_snow_free_9', 'nearnad_snow_free_10',
       'offnad_snow_cov_1', 'offnad_snow_cov_2', 'offnad_snow_cov_3',
       'offnad_snow_cov_4', 'offnad_snow_cov_5', 'offnad_snow_cov_6',
       'offnad_snow_cov_7', 'offnad_snow_cov_8', 'offnad_snow_cov_9',
       'offnad_snow_cov_10', 'offnad_snow_free_1', 'offnad_snow_free_2',
       'offnad_snow_free_3', 'offnad_snow_free_4', 'offnad_snow_free_5',
       'offnad_snow_free_6', 'offnad_snow_free_7', 'offnad_snow_free_8',
       'offnad_snow_free_9', 'offnad_snow_free_10', 'al

In [32]:
# train a random forest model
from sklearn.ensemble import RandomForestRegressor

# drop year and gdp columns
X = data.drop(columns=["year", "gdp"])
y = data["gdp"]

# standardise gdp
y_mean = y.mean()
y_std = y.std()
y = (y - y_mean) / y_std

# take randomly 80% of the data for training
train_size = int(0.8 * len(data))
test_size = len(data) - train_size

train_indices = np.random.choice(len(data), train_size, replace=False)
test_indices = np.setdiff1d(np.arange(len(data)), train_indices)

X_train = X.iloc[train_indices]
y_train = y.iloc[train_indices]
X_test = X.iloc[test_indices]
y_test = y.iloc[test_indices]

# train the model
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(X_train, y_train)

# get the predictions
y_hat = model.predict(X_test)

# compute the mae
mae = np.mean(np.abs(y_test - y_hat))

# compute the mean percentage error
percentage_error = np.mean(100*np.abs((y_test - y_hat) / y_test))

In [34]:
print(mae)
print(percentage_error)

0.13538272435718499
40.682541294026976


In [37]:
# fit a lasso regression model 
from sklearn.linear_model import Lasso

model = Lasso(alpha=0)

model.fit(X_train, y_train)

# get the predictions
y_hat = model.predict(X_test)

# compute the mae
mae = np.mean(np.abs(y_test - y_hat))

# compute the mean percentage error
percentage_error = np.mean(100*np.abs((y_test - y_hat) / y_test))

print(mae)
print(percentage_error)


0.1317128785693757
37.73945120840673


C:\Users\jakub\AppData\Local\Temp/ipykernel_18660/1922239168.py:6: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  model.fit(X_train, y_train)
c:\Users\jakub\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
c:\Users\jakub\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.133629836587964, tolerance: 0.02335093979204034
  model = cd_fast.enet_coordinate_descent(


In [40]:
# fit a XGBoost model
import xgboost as xgb

model = xgb.XGBRegressor(objective="reg:squarederror", random_state=0)

model.fit(X_train, y_train)

# get the predictions
y_hat = model.predict(X_test)

# compute the mae
mae = np.mean(np.abs(y_test - y_hat))

# compute the mean percentage error
percentage_error = np.mean(100*np.abs((y_test - y_hat) / y_test))

print(mae)
print(percentage_error)

0.20718964193497746
61.93365225125984


In [47]:
# fit a neural network model
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Resizing, Dropout, BatchNormalization
from keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# turn X_train bool to integers
boolean_columns = [col for col in X_train.columns if X_train[col].dtype == bool]

for col in boolean_columns:
    X_train[col] = X_train[col].astype(int) 
 

# fit the model on the X_train and y_train
model = Sequential()
model.add(Dense(256, activation="relu", input_dim=X_train.shape[1]))  # More complex first layer
model.add(Dropout(0.3))  # Dropout to reduce overfitting
model.add(Dense(128, activation="relu"))  # Second layer
model.add(Dropout(0.2))  # Additional Dropout layer
model.add(Dense(64, activation="relu"))  # Third layer
model.add(Dense(32, activation="relu"))  # Fourth layer
model.add(Dropout(0.1))  # Dropout layer
model.add(Dense(16, activation="relu"))  # New additional layer
model.add(Dense(1)) 

model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])

model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# get the predictions
boolean_columns = [col for col in X_test.columns if X_test[col].dtype == bool]
for col in boolean_columns:
    X_test[col] = X_test[col].astype(int)

y_hat = model.predict(X_test).flatten()

# compute the mae
mae = np.mean(np.abs(y_test - y_hat))

# compute the mean percentage error
percentage_error = np.mean(100*np.abs((y_test - y_hat) / y_test))

print(mae)
print(percentage_error)

Epoch 1/100
5/5 [==============================] - 2s 83ms/step - loss: 0.9688 - mae: 0.6054 - val_loss: 1.8431 - val_mae: 0.7224
Epoch 2/100
5/5 [==============================] - 0s 21ms/step - loss: 0.8611 - mae: 0.5608 - val_loss: 1.6119 - val_mae: 0.6718
Epoch 3/100
5/5 [==============================] - 0s 18ms/step - loss: 0.7547 - mae: 0.5229 - val_loss: 1.3014 - val_mae: 0.5832
Epoch 4/100
5/5 [==============================] - 0s 17ms/step - loss: 0.5846 - mae: 0.4480 - val_loss: 0.9138 - val_mae: 0.4808
Epoch 5/100
5/5 [==============================] - 0s 17ms/step - loss: 0.4374 - mae: 0.3808 - val_loss: 0.4224 - val_mae: 0.3318
Epoch 6/100
5/5 [==============================] - 0s 16ms/step - loss: 0.1964 - mae: 0.3024 - val_loss: 0.1793 - val_mae: 0.3302
Epoch 7/100
5/5 [==============================] - 0s 17ms/step - loss: 0.1813 - mae: 0.3020 - val_loss: 0.1219 - val_mae: 0.2604
Epoch 8/100
5/5 [==============================] - 0s 17ms/step - loss: 0.1546 - mae: 0.23

In [42]:
X_train

,nearnad_snow_cov_1,nearnad_snow_cov_2,nearnad_snow_cov_3,nearnad_snow_cov_4,nearnad_snow_cov_5,nearnad_snow_cov_6,nearnad_snow_cov_7,nearnad_snow_cov_8,nearnad_snow_cov_9,nearnad_snow_cov_10,...,region_Odessa_Oblast,region_Poltava_Oblast,region_Rivne_Oblast,region_Sumy_Oblast,region_Ternopil_Oblast,region_Vinnytsia_Oblast,region_Volyn_Oblast,region_Zakarpattia_Oblast,region_Zaporizhia_Oblast,region_Zhytomyr_Oblast
37,0.0,0.495924,0.339998,0.101624,0.041719,0.012099,0.007851,0.000602,0.000140,0.000043,...,False,False,False,False,False,False,False,False,False,False
176,0.0,0.826179,0.126032,0.019672,0.023089,0.003834,0.001195,0.000000,0.000000,0.000000,...,False,False,True,False,False,False,False,False,False,False
116,0.0,0.643525,0.207011,0.079590,0.052258,0.011659,0.005083,0.000450,0.000219,0.000206,...,False,False,False,False,False,False,False,False,False,False
7,0.0,0.615698,0.295872,0.063118,0.020931,0.002426,0.001467,0.000163,0.000127,0.000199,...,False,False,False,False,False,False,False,False,False,False
54,0.0,0.674845,0.255918,0.044521,0.019125,0.004416,0.001071,0.000105,0.000000,0.000000,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,0.0,0.537389,0.333745,0.080644,0.042664,0.003911,0.001132,0.000051,0.000309,0.000154,...,False,False,False,False,False,False,False,False,False,False
60,0.0,0.611959,0.247615,0.090376,0.034844,0.013385,0.001498,0.000262,0.000062,0.000000,...,False,False,False,False,False,False,False,False,False,False
120,0.0,0.541128,0.297913,0.100557,0.039343,0.016470,0.004401,0.000189,0.000000,0.000000,...,False,False,False,False,False,False,False,False,False,False
207,0.0,0.675439,0.238675,0.062634,0.018674,0.002677,0.001807,0.000027,0.000000,0.000067,...,False,False,False,False,False,True,False,False,False,False


In [43]:
y_train

37     1.447782
176   -0.530288
116    0.349765
7     -0.319760
54    -0.412693
         ...   
8     -0.352376
60     0.533187
120    0.123711
207   -0.145052
117    0.401096
Name: gdp, Length: 200, dtype: float64